In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

c:\ANACON~1\envs\torch\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
c:\ANACON~1\envs\torch\lib\site-packages\numpy\.libs\libopenblas.QVLO2T66WEPI7JZ63PS3HMOHFEY472BC.gfortran-win_amd64.dll
c:\ANACON~1\envs\torch\lib\site-packages\numpy\.libs\libopenblas.XWYDX2IKJW2NMTWSFYNGFUWKQU3LYTCZ.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [2]:
def get_data(URL):
    res = requests.get(URL)
    soup = BeautifulSoup(res.content, 'html.parser')
    rows = soup.find_all('div', attrs={'class': 'row mb-2'})
    result_list = []
    for row in rows:
        accepted = row.find('span', attrs={'class': 'badge badge-green'})
        if accepted is not None:
            accepted_text = accepted.text.strip()
            # remove new line
            accepted_text = accepted_text.replace('\n', ' ')
            # remove excess space in the middle
            accepted_text = ' '.join(accepted_text.split())
            acceptance_date = '-'.join(accepted_text.split(' ')[-2:])
            result_list.append({
                'date': acceptance_date,
                'status': 'accepted'
            })
        else:
            rejected = row.find('span', attrs={'class': 'badge badge-red'})
            if rejected is not None:
                rejected_text = rejected.text.strip()
                # remove new line
                rejected_text = rejected_text.replace('\n', ' ')
                # remove excess space in the middle
                rejected_text = ' '.join(rejected_text.split())
                rejection_date = '-'.join(rejected_text.split(' ')[-2:])
                result_list.append({
                    'date': rejection_date,
                    'status': 'rejected'
                })

    return result_list

In [3]:
all_result_list = []
for _page in range(1, 5):
    try:
        URL = f"https://www.thegradcafe.com/survey/?institution=Georgia+Institute+of+Technology&program=Computer+Science&degree=Masters&season=F22&page={_page}&per_page=40"
        result_list = get_data(URL)
        all_result_list.extend(result_list)
    except:
        pass

In [4]:
df = pd.DataFrame(all_result_list)

In [5]:
df

,date,status
0,25-Apr,accepted
1,8-Apr,rejected
2,8-Apr,rejected
3,8-Apr,rejected
4,8-Apr,rejected
5,8-Apr,rejected
6,8-Apr,rejected
7,8-Apr,rejected
8,8-Apr,rejected
9,8-Apr,rejected


In [6]:
# convert date to datetime
# date if form of '9-Apr', '17-Feb' and year is 2022
df['date'] = pd.to_datetime(df['date'] + '-2022', format='%d-%b-%Y')

In [7]:
import plotly.graph_objects as go

In [8]:
# create a figure
fig = go.Figure()

# make bar plot, x is date, y is count of accepts and rejects on that date
fig.add_trace(go.Bar(x=df[df['status'] == 'accepted']['date'].value_counts().index,
                        y=df[df['status'] == 'accepted']['date'].value_counts().values, 
                        name='Accepted'))

fig.add_trace(go.Bar(x=df[df['status'] == 'rejected']['date'].value_counts().index,
                        y=df[df['status'] == 'rejected']['date'].value_counts().values,                     
                        name='Rejected'))

fig.show()